In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sb
import statsmodels.formula.api as smf
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf 

ModuleNotFoundError: No module named 'pandas.util'

In [ ]:
cc=pd.read_excel('/Users/pratikgade123/Desktop/Pandas/CocaCola_Sales_Rawdata.xlsx')

In [ ]:
cc

In [ ]:
cc.info()

In [ ]:
cc.shape

In [ ]:
plt.title("Line Plot", size = 15, weight = 'bold')
plt.ylabel("Sales", size = 10, weight = 'bold')
plt.plot(cc['Sales'])

In [ ]:
# Boxplot for ever
sb.boxplot("Sales",data = cc)

In [ ]:
sb.factorplot("Quarter","Sales",data=cc,kind="box")

In [ ]:
quarters = ['a','b','c','d']
n = cc['Quarter'][0]
n[0:2]

In [ ]:
cc['quarter'] = 0
for i in range(42):
    n = cc['Quarter'][i]
    cc['quarter'][i] = n[0:2]

In [ ]:
dummy = pd.DataFrame(pd.get_dummies(cc['quarter']))


In [ ]:
dummy


In [ ]:
cc_c=pd.concat([cc,dummy],axis=1)


In [ ]:
cc_c

In [ ]:
cc_c["t"] = np.arange(1,43)


In [ ]:
cc_c["t_squared"] = cc_c["t"]*cc_c["t"]
cc_c.columns
cc_c["log_Sales"] = np.log(cc_c["Sales"])


In [ ]:
cc_c

In [ ]:
plt.figure(figsize=(12,8))
heatmap_y_quarter = pd.pivot_table(data=cc_c,values="Sales",index="Quarter",columns="quarter",aggfunc="mean", fill_value = 0)
sb.heatmap(heatmap_y_quarter,annot=True,fmt="g") #fmt is format of the grid values

In [ ]:
# Boxplot for ever
plt.figure(figsize=(8,6))
sb.boxplot(x="quarter",y="Sales",data=cc_c)

In [ ]:
lag_plot(cc_c['Sales'])
plt.title("Lag Plot", size = 15, weight = "bold")
plt.show()

In [ ]:
plot_acf(cc_c['Sales'], lags = 35, color = 'green')               
plt.xlabel("No of lags, k = 35", size = 10, weight = 'bold')
plt.ylabel("Autocorrelation (r2 value)", size = 20, weight = 'bold')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sb.lineplot(x="quarter",y="Sales",data=cc_c)

In [ ]:
train =cc_c.head(33)
test  =cc_c.tail(7)

In [ ]:
train

In [ ]:
test

In [ ]:
import statsmodels.formula.api as smf 

In [ ]:
#linear Model

In [ ]:
linear = smf.ols('Sales~t',data=train).fit()
pred_linear =  pd.Series(linear.predict(pd.DataFrame(test['t'])))
rms_linear = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_linear))**2))
rms_linear

In [ ]:
Quad = smf.ols('Sales~t+t_squared',data=train).fit()                             #quadratic model
pred_Quad = pd.Series(Quad.predict(test[["t","t_squared"]]))
rms_Quad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_Quad))**2))
print("RMS Error : ",rms_Quad)

In [ ]:
Exp = smf.ols('log_Sales~t',data=train).fit()                                    #exponential model
pred_Exp = pd.Series(Exp.predict(pd.DataFrame(test['t'])))
rms_Exp = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Exp)))**2))
print("RMS Error : ",rms_Exp)

In [ ]:
add_sea = smf.ols('Sales~Q1+Q2+Q3+Q4',data=train).fit()                          #additive seasonality model
pred_add_sea = pd.Series(add_sea.predict(test[['Q1','Q2','Q3','Q4']]))
rms_add_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_add_sea))**2))
print("RMS Error : ",rms_add_sea)


In [ ]:

add_sea_quad = smf.ols('Sales~t+t_squared+Q1+Q2+Q3+Q4',data=train).fit()         #additive seasonality qudratic model
pred_add_sea_quad = pd.Series(add_sea_quad.predict(test[['t','t_squared','Q1','Q2','Q3','Q4']]))
rms_add_sea_quad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_add_sea_quad))**2))
print("RMS Error : ",rms_add_sea_quad)

In [ ]:
Mul_sea = smf.ols('log_Sales~Q1+Q2+Q3+Q4',data = train).fit()                     #multiplicative seasonality model
pred_Mult_sea = pd.Series(Mul_sea.predict(test))
rms_Mult_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Mult_sea)))**2))
print("RMS Error : ",rms_Mult_sea)

In [ ]:
Mul_Add_sea = smf.ols('log_Sales~t+Q1+Q2+Q3+Q4',data = train).fit() #multiplicative additive seasonality
pred_Mult_add_sea = pd.Series(Mul_Add_sea.predict(test))
rms_Mult_add_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Mult_add_sea)))**2))
print("RMS Error : ",rms_Mult_add_sea) 

In [ ]:
#tabuling rmes value
Final_data = {"MODEL":pd.Series(["rms_linear","rms_Exp","rms_Quad","rms_add_sea","rms_add_sea_quad","rms_Mult_sea","rms_Mult_add_sea"]),"RMS_Values":pd.Series([rms_linear,rms_Exp,rms_Quad,rms_add_sea,rms_add_sea_quad,rms_Mult_sea,rms_Mult_add_sea])}
Final_r = pd.DataFrame(Final_data)

In [ ]:
Final_r.sort_values(['RMS_Values'])

In [ ]:
data_precict=pd.read_excel('/Users/pratikgade123/Desktop/Pandas/CocaCola_Sales_Rawdata.xlsx')

In [ ]:
Final_pred = smf.ols('log_Sales~t+Q1+Q2+Q3+Q4',data=cc_c).fit()

In [ ]:
pred_latest  = pd.Series(Final_pred.predict(cc_c))

In [ ]:
pred_latest

In [ ]:
from pandas_profiling import ProfileReport
PR=ProfileReport(cc)
PR

In [ ]:
airline= pd.read_excel("/Users/pratikgade123/Desktop/Pandas/Airlines+Data.xlsx")

In [ ]:
airline

In [ ]:
airline.info()

In [ ]:
airline.shape

In [ ]:
# Visualizing the overall data 

In [ ]:

plt.title("Line Plot", size = 20, weight = 'bold')
plt.ylabel("Passengers", size = 15, weight = 'bold')
plt.plot(airline['Passengers'])

In [ ]:
airline["month"] = airline.Month.dt.strftime("%b")

In [ ]:
airline

In [ ]:
data = pd.get_dummies(airline["month"])

In [ ]:
data

In [ ]:
new = pd.concat([airline,data],axis=1)

In [ ]:
#find t_squared values and log values
new["t"] = np.arange(1,97)
new["t_squared"] = new["t"]*new["t"]
new.columns
new["log_passengers"] = np.log(new["Passengers"])

In [ ]:
new

In [ ]:
train= new.head(75)
test=new.tail(21)
new.Passengers.plot()


In [ ]:
indexed = new.set_index(['Month'])
indexed.head(5)

In [ ]:
rolmean = indexedDataset.rolling(window=12).mean() 
#window size 12 denotes 1 Year

rolstd = indexedDataset.rolling(window=12).std()
print(rolmean,rolstd)

In [ ]:
new.Passengers.plot(label="org")
for i in range(2,10,2):
    new["Passengers"].rolling(i).mean().plot(label=str(i))
plt.legend(loc=4)

In [ ]:
#ACF plot and PCAF plot

In [ ]:
import statsmodels.graphics.tsaplots as tsa_plots
tsa_plots.plot_acf(new.Passengers,lags=12)
tsa_plots.plot_pacf(new.Passengers,lags=12)
plt.show()

In [ ]:
Quadratic Model

In [ ]:

quad=smf.ols('Passengers~t+t_squared',data=new).fit()
predquad=pd.Series(quad.predict(pd.DataFrame(test[['t','t_squared']])))
rmsequad=np.sqrt(np.mean((np.array(test['Passengers'])-np.array(predquad))**2))
print("RMS Error : ",rmsequad)

In [ ]:
linear model

In [ ]:

linear= smf.ols('Passengers~t',data=new).fit()
predlin=pd.Series(linear.predict(pd.DataFrame(test['t'])))
rmselin=np.sqrt((np.mean(np.array(test['Passengers'])-np.array(predlin))**2))
print("RMS Error : ",rmselin)

In [ ]:
#exponential model

In [ ]:

expo=smf.ols('log_passengers~t',data=new).fit()
predexp=pd.Series(expo.predict(pd.DataFrame(test['t'])))
predexp
rmseexpo=np.sqrt(np.mean((np.array(test['Passengers'])-np.array(np.exp(predexp)))**2))
print("RMS Error : ",rmseexpo)

In [ ]:
#additive seasonality

In [ ]:
additive= smf.ols('Passengers~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data=new).fit()
predadd=pd.Series(additive.predict(pd.DataFrame(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']])))
predadd
rmseadd=np.sqrt(np.mean((np.array(test['Passengers'])-np.array(predadd))**2))
print("RMS Error : ",rmseadd)

In [ ]:
#additive seasonality with linear trend
addlinear= smf.ols('Passengers~t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data=new).fit()
PAL=pd.Series(addlinear.predict(pd.DataFrame(test[['t','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']])))
PAL

In [ ]:
rmseaddlinear=np.sqrt(np.mean((np.array(test['Passengers'])-np.array(PAL))**2))
print("RMS Error : ",rmseaddlinear)

In [ ]:
#additive seasonality with quadratic trend
addquad=smf.ols('Passengers~t+t_squared+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data=new).fit()
predaddquad=pd.Series(addquad.predict(pd.DataFrame(test[['t','t_squared','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']])))
rmseaddquad=np.sqrt(np.mean((np.array(test['Passengers'])-np.array(PAL))**2))
print("Root Mean Square Error : ",rmseaddquad)

In [ ]:
#multiplicative seasonality
mulsea=smf.ols('log_passengers~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data=new).fit()
predmul= pd.Series(mulsea.predict(pd.DataFrame(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']])))
rmsemul= np.sqrt(np.mean((np.array(test['Passengers'])-np.array(np.exp(predmul)))**2))
print("Root Mean Square Error : ",rmsemul)

In [ ]:
#multiplicative seasonality with quadratic trend
mul_quad= smf.ols('log_passengers~t+t_squared+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data=new).fit()
pred_mul_quad= pd.Series(mul_quad.predict(test[['t','t_squared','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]))
rmse_mul_quad=np.sqrt(np.mean((np.array(test['Passengers'])-np.array(np.exp(pred_mul_quad)))**2))
print("Root Mean Square Error : ",rmse_mul_quad)

In [ ]:
data={'Model':pd.Series(['rmse_mul_quad','rmseadd','rmseaddlinear','rmseaddquad','rmseexpo','rmselin','rmsemul','rmsequad']),'Values':pd.Series([rmse_mul_quad,rmseadd,rmseaddlinear,rmseaddquad,rmseexpo,rmselin,rmsemul,rmsequad])}


In [ ]:
RMSE=pd.DataFrame(data)
RMSE

In [ ]:
#import dataset for making model

In [ ]:
Final= pd.read_excel("/Users/pratikgade123/Desktop/Pandas/Airlines+Data.xlsx")

In [ ]:
Final_pred = smf.ols('log_passengers~t+t_squared+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data=new).fit()

In [ ]:
pred_final  = pd.Series(Final_pred.predict(new))

In [ ]:
pred_final